In [1]:
if "running_all" not in globals():
    from ipywidgets import widgets
    matrices = ["ex10", "msc04515", "s1rmq4m1", "Na5", "bcsstk18",
                "vibrobox", "cbuckle", "Pres_Poisson", "raefsky4", "vanbody",
                "ct20stif", "cant", "bcircuit", "apache1",
                "consph"] # , "parabolic_fem", "bmw7st_1", "tmt_sym", "gridgena", "cfd2"]
    b = widgets.Button(description="Run over all matrices", button_style="success")
    output = widgets.Output()

    display(b, output)

    def run_over_all_matrices(button):
        global running_all
        global matrix
        running_all = True
        with output:
            for matrix in matrices:
                print(f"Running {matrix}...")
                %run ./protection_vis.ipynb # will output at this cell rather than later
            print("Finished!")
    b.on_click(run_over_all_matrices)

Button(button_style='success', description='Run over all matrices', style=ButtonStyle())

Output()

In [ ]:
if "running_all" not in globals():
    matrix = "gridgena"  # manually set to run over one matrix
matrix

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
from json import load
sys.path.append(os.path.join(os.getcwd(), os.pardir))
from io_utils import load_matrices_from_dir, load_models

In [ ]:
mats = load_matrices_from_dir("../matrices/raw", subset=[matrix])
mat = list(mats.values())[0]
mat

In [ ]:
mat_errorfree_iters = {
    "apache1": 131, 
    "bcircuit": 48, 
    "bcsstk18": 432, 
    "bmw7st_1": 597, 
    "cant": 394,
    "cbuckle": 341, 
    "cfd2": 840, 
    "consph": 611, 
    "ct20stif": 364, 
    "ex10": 479,
    "gridgena": 1434, 
    "msc04515": 1849, 
    "Na5": 101, 
    "parabolic_fem": 126, 
    "Pres_Poisson": 72,
    "raefsky4": 79, 
    "s1rmq4m1": 605, 
    "tmt_sym": 237, 
    "vanbody": 491, 
    "vibrobox": 565,
}
errorfree_iterations = mat_errorfree_iters[matrix]

In [ ]:
chosen_models = {
    "apache1": "XGBRegressor",
    "bcircuit": "XGBRegressor",
    "bcsstk18": "XGBRegressor",
    "bmw7st_1": "XGBRegressor",
    "cant": "RandomForestRegressor",
    "cbuckle": "RandomForestRegressor",
    "cfd2": "Ridge",
    "consph": "Ridge",
    "ct20stif": "XGBRegressor",
    "ex10": "RandomForestRegressor",
    "gridgena": "Ridge",
    "msc04515": "RandomForestRegressor",
    "Na5": "XGBRegressor",
    "parabolic_fem": "Ridge",
    "Pres_Poisson": "XGBRegressor",
    "raefsky4": "RandomForestRegressor",
    "s1rmq4m1": "RandomForestRegressor",
    "tmt_sym": "RandomForestRegressor",
    "vanbody": "KNeighborsRegressor",
    "vibrobox": "KNeighborsRegressor",
}
model_path = f"./models/{matrix}/best_{chosen_models[matrix]}.pkl"
model = load_models([model_path])[0]
model

In [ ]:
with open(f"../matrices/2norms/{matrix}_pos_2norms.json") as f:
    pos_2norms = load(f)

In [ ]:
df = pd.DataFrame([[i, pos_2norms[str(pos)], pos] for pos in range(mat.shape[0])
                   for i in range(errorfree_iterations)], columns=["i", "2norm", "rowid"])
len(df)

In [ ]:
df["output"] = model.predict(df[["i", "2norm"]])
df.head()

In [ ]:
max_2norm = df["2norm"].max()
spaced_2norms = np.geomspace(1e-6, max_2norm, num=1000)
iter_norm_pairs_spaced = np.array([[i, n, p] for p, n in enumerate(spaced_2norms)
                                       for i in range(errorfree_iterations)])
df_spaced = pd.DataFrame(iter_norm_pairs_spaced, columns=["i", "2norm", "rowid"])
df_spaced["output"] = model.predict(df_spaced[["i", "2norm"]])
df_spaced.head()

In [ ]:
df_spaced_piv = df_spaced.pivot(index=["2norm", "rowid"], columns="i", values="output")

protections = df_spaced_piv.to_numpy()
df_spaced_piv.head()

In [ ]:
plt.gcf().set_size_inches(8, 8)
plt.gcf().set_dpi(100)

ax = sns.heatmap(df_spaced_piv, vmin=1, vmax=df_spaced_piv.max().max())

plt.title("Predicted Outputs by Iteration and Row 2-norm")
plt.xlabel("Iteration Number")
plt.ylabel("Corresponding Row 2-norm (log scale)")
plt.savefig(f"./figures/{matrix}/iter_r2n_output.png")
plt.show()

In [ ]:
df_piv = df.pivot(index=["rowid"], columns="i", values="output")
df_piv.head()

In [ ]:
plt.gcf().set_size_inches(8, 8)
plt.gcf().set_dpi(100)
sns.heatmap(df_piv, vmin=0, vmax=df_piv.max().max())
plt.title("Protection Scheme")
plt.xlabel("Iteration Number")
plt.ylabel("Row Index")
plt.savefig(f"./figures/{matrix}/iter_row_output.png")
plt.show()